## Introdução

Este projeto é o desafio técnico do processo seletivo da Indicium para o programa LightHouse, na trilha de Ciência de Dados.

Para esse notebook, será desenvolvido uma análise exploratória do dataset `desafio_indicium_imdb` para entender sobre a estrutura dos dados, analisar cada features, identificar a coluna "alvo", preparar os dados e criar um modelo de regressão com `sckit-learn`.

**Informações do Dataset:**
- `Series_Title`: Nome do filme
- `Released_Year`: Ano de lançamento
- `Certificate`: Classificação etária
- `Runtime`: Tempo de duração
- `Genre`: Gênero
- `IMDB_Rating`: Nota do IMDB
- `Overview`: Overview do filme
- `Meta_score`: Média ponderada de todas as críticas 
- `Director`: Diretor
- `Star1`: Ator/atriz #1
- `Star2`: Ator/atriz #2
- `Star3`: Ator/atriz #3
- `Star4`: Ator/atriz #4
- `No_of_Votes`: Número de votos
- `Gross`: Faturamento

### Carregando bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carregando os dados e inspeção inicial

In [2]:
df = pd.read_csv("../data/desafio_indicium_imdb.csv")

In [3]:
print(f"Quantidade de linhas: {df.shape[0]}")
print(f"Quantidade de colunas: {df.shape[1]}")

Quantidade de linhas: 999
Quantidade de colunas: 16


In [4]:
# Visão geral do Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     999 non-null    int64  
 1   Series_Title   999 non-null    object 
 2   Released_Year  999 non-null    object 
 3   Certificate    898 non-null    object 
 4   Runtime        999 non-null    object 
 5   Genre          999 non-null    object 
 6   IMDB_Rating    999 non-null    float64
 7   Overview       999 non-null    object 
 8   Meta_score     842 non-null    float64
 9   Director       999 non-null    object 
 10  Star1          999 non-null    object 
 11  Star2          999 non-null    object 
 12  Star3          999 non-null    object 
 13  Star4          999 non-null    object 
 14  No_of_Votes    999 non-null    int64  
 15  Gross          830 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 125.0+ KB


Existem algumas features inconsistentens nos dados, como uma coluna sem nome; valores `non-null` abaixo da quantidade de linhas (indicando valores ausentes) nas colunas `Certificate`, `Meta_score`, `Gross`; algumas colunas com Dtype no formato `object` que possívelmente deveria estar como `int` ou `float`.

In [5]:
# Primeiras linhas do Dataframe
df.head()

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
1,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
2,3,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
3,4,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
4,5,The Lord of the Rings: The Return of the King,2003,U,201 min,"Action, Adventure, Drama",8.9,Gandalf and Aragorn lead the World of Men agai...,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,1642758,"377,845,905"


### Limpeza dos Dados

Nesta etapa, será removida a coluna "Unnamed" pois se trata apenas de um índice que foi carregado junto aos dados. Em seguida, vamos lidar com duplicatas se necessário e com os valores ausentes.

In [6]:
# Removendo colunas desnecessárias
df.drop(columns="Unnamed: 0", inplace=True)

In [7]:
print("Contagem de filmes com o mesmo nome:")
df['Series_Title'].value_counts().head()

Contagem de filmes com o mesmo nome:


Series_Title
Drishyam            2
Falling Down        1
Cidade de Deus      1
The Muppet Movie    1
The Warriors        1
Name: count, dtype: int64

Existe apenas um filme com duplicata, o que não irá prejudicar as análises. Um outro ponto importante a se fazer inicialmente, será converter as seguintes colunas para `int`:

- Released_Year

- Runtime

- Gross (a decisão de nao usar float, se deu ao fato de desconsiderar as moedas)

Para isso, será necessário a utilização de técnicas específicas durante a conversão de cada uma dessas colunas.

In [8]:
# Ano de estréia para int
df["Released_Year"] = pd.to_numeric(df['Released_Year'], downcast='integer', errors='coerce')

Os parâmetros `downcast` garante a conversão direta para inteiro e `errors` como 'coerce' irá retornar a data para NaN se a conversão não for possível (por haver um caractére especial ou letra). 

In [9]:
# Duração para int
df["Runtime"] = df['Runtime'].str.replace(' min', '')
df["Runtime"] = pd.to_numeric(df['Runtime'], downcast='integer', errors='coerce')

In [10]:
# Faturamento para int
df["Gross"] = df['Gross'].str.replace(r'[\$,]', '', regex=True)
df["Gross"] = pd.to_numeric(df['Gross'], downcast='integer', errors='coerce')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   999 non-null    object 
 1   Released_Year  998 non-null    float64
 2   Certificate    898 non-null    object 
 3   Runtime        999 non-null    int16  
 4   Genre          999 non-null    object 
 5   IMDB_Rating    999 non-null    float64
 6   Overview       999 non-null    object 
 7   Meta_score     842 non-null    float64
 8   Director       999 non-null    object 
 9   Star1          999 non-null    object 
 10  Star2          999 non-null    object 
 11  Star3          999 non-null    object 
 12  Star4          999 non-null    object 
 13  No_of_Votes    999 non-null    int64  
 14  Gross          830 non-null    float64
dtypes: float64(4), int16(1), int64(1), object(9)
memory usage: 111.3+ KB


In [12]:
print("Contagem de valores nulos por coluna:")
df.isnull().sum()

Contagem de valores nulos por coluna:


Series_Title       0
Released_Year      1
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

As colunas `Released_Year`, `Certificate`, `Meta_score` e `Gross` possuem valores ausentes que precisam de atenção. Mas primeiro, é necessário observar esses dados de perto. Para isso, será usada uma máscara lógica para observar as amostras e decidir a melhor maneira de lidar com esses valores.

In [13]:
# Criando uma máscara lógica com NaNs   
colunas_null = ["Released_Year", "Certificate", "Meta_score", "Gross"]
null_mask = df[colunas_null].isnull()
null_mask[0:5]

,Released_Year,Certificate,Meta_score,Gross
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False


In [14]:
# Filtragem com cinco amostras
linhas_com_null = null_mask.any(axis=1)
df[linhas_com_null].head()

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
17,Hamilton,2020.0,PG-13,160,"Biography, Drama, History",8.6,The real life of one of America's foremost fou...,90.0,Thomas Kail,Lin-Manuel Miranda,Phillipa Soo,Leslie Odom Jr.,Renée Elise Goldsberry,55291,NaN
19,Soorarai Pottru,2020.0,U,153,Drama,8.6,"Nedumaaran Rajangam ""Maara"" sets out to make t...",NaN,Sudha Kongara,Suriya,Madhavan,Paresh Rawal,Aparna Balamurali,54995,NaN
29,Seppuku,1962.0,NaN,133,"Action, Drama, Mystery",8.6,When a ronin requesting seppuku at a feudal lo...,85.0,Masaki Kobayashi,Tatsuya Nakadai,Akira Ishihama,Shima Iwashita,Tetsurô Tanba,42004,NaN
31,It's a Wonderful Life,1946.0,PG,130,"Drama, Family, Fantasy",8.6,An angel is sent from Heaven to help a despera...,89.0,Frank Capra,James Stewart,Donna Reed,Lionel Barrymore,Thomas Mitchell,405801,NaN
45,Hotaru no haka,1988.0,U,89,"Animation, Drama, War",8.5,A young boy and his little sister struggle to ...,94.0,Isao Takahata,Tsutomu Tatsumi,Ayano Shiraishi,Akemi Yamaguchi,Yoshiko Shinohara,235231,NaN


Os valores ausentes irão impactar negativamente o desempenho das analises, principalmente a quantidade de NaNs em `Meta_score` e `Gross`. A melhor decisão, será remover esses valores.

In [15]:
df.dropna(inplace=True)

In [16]:
print(f"Quantidade de linhas após remoção de NaNs: {df.shape[0]}\nValores nulos restantes:")
df.isna().sum()

Quantidade de linhas após remoção de NaNs: 712
Valores nulos restantes:


Series_Title     0
Released_Year    0
Certificate      0
Runtime          0
Genre            0
IMDB_Rating      0
Overview         0
Meta_score       0
Director         0
Star1            0
Star2            0
Star3            0
Star4            0
No_of_Votes      0
Gross            0
dtype: int64